In [4]:
import gymnasium as gym
from gymnasium.wrappers import RecordVideo
from stable_baselines3 import PPO
from stable_baselines3.common.env_util import make_vec_env
from stable_baselines3.common.vec_env import SubprocVecEnv

In [8]:
TRAIN = True

if True : 
    n_cpu = 6
    batch_size = 64
    env = make_vec_env("racetrack-v0", n_envs=n_cpu, vec_env_cls=SubprocVecEnv)
    model = PPO(
        "MlpPolicy",
        env,
        policy_kwargs=dict(net_arch=[dict(pi=[256, 256], vf=[256, 256])]),
        n_steps=batch_size * 12 // n_cpu,
        batch_size=batch_size,
        n_epochs=10,
        learning_rate=5e-4,
        gamma=0.9,
        verbose=2,
        tensorboard_log="racetrack_ppo/",
    )
    # Train the model
    if TRAIN:
        model.learn(total_timesteps=int(1e5))
        model.save("racetrack_ppo/model")
        del model

Using cpu device


/Users/BOULLIERERWAN1/opt/anaconda3/envs/rl_py311/lib/python3.11/site-packages/stable_baselines3/common/policies.py:486: UserWarning: As shared layers in the mlp_extractor are removed since SB3 v1.8.0, you should now pass directly a dictionary and not a list (net_arch=dict(pi=..., vf=...) instead of net_arch=[dict(pi=..., vf=...)])
  warnings.warn(


Logging to racetrack_ppo/PPO_9
---------------------------------
| rollout/           |          |
|    ep_len_mean     | 27       |
|    ep_rew_mean     | 2.13     |
| time/              |          |
|    fps             | 73       |
|    iterations      | 1        |
|    time_elapsed    | 10       |
|    total_timesteps | 768      |
---------------------------------
-----------------------------------------
| rollout/                |             |
|    ep_len_mean          | 27          |
|    ep_rew_mean          | 2.13        |
| time/                   |             |
|    fps                  | 67          |
|    iterations           | 2           |
|    time_elapsed         | 22          |
|    total_timesteps      | 1536        |
| train/                  |             |
|    approx_kl            | 0.017712874 |
|    clip_fraction        | 0.085       |
|    clip_range           | 0.2         |
|    entropy_loss         | -1.4        |
|    explained_variance   | -0.206      |

ValueError: Render mode is None, which is incompatible with RecordVideo. Initialize your environment with a render_mode that returns an image, such as rgb_array.

In [12]:
# Run the algorithm
model = PPO.load("racetrack_ppo/model", env=env)

env = gym.make("racetrack-v0", render_mode="rgb_array")
env = RecordVideo(
    env, video_folder="racetrack_ppo/videos", episode_trigger=lambda e: True
)
env.unwrapped.set_record_video_wrapper(env)

for video in range(10):
    done = truncated = False
    obs, info = env.reset()
    while not (done or truncated):
        # Predict
        action, _states = model.predict(obs, deterministic=True)
        # Get reward
        obs, reward, done, truncated, info = env.step(action)
        # Render
        env.render()
env.close()

Wrapping the env with a `Monitor` wrapper
Wrapping the env in a DummyVecEnv.


AttributeError: 'NoneType' object has no attribute 'get_image'

: 

In [2]:
import numpy as np
import gymnasium as gym

env = gym.make("racetrack-v0", render_mode="rgb_array")

config = {
    "observation": {
        "type": "OccupancyGrid",
        "features": ["presence", "on_road"],
        "grid_size": [[-18, 18], [-18, 18]],
        "grid_step": [5, 5],
        "as_image": False,
        "align_to_vehicle_axes": True
    },
    "action": {
        "type": "ContinuousAction",
        "longitudinal": False,
        "lateral": True,
        "steering_range": [-np.pi / 4, np.pi / 4],  # [rad]
        #"acceleration_range": [-2, 2],  # [m/s²]
        "speed_range": [0, 15],  # [m/s]
    },
    "simulation_frequency": 15,
    "policy_frequency": 5,
    "duration": 300,
    "collision_reward": -1,
    "lane_centering_cost": 4,
    "action_reward": -0.3,
    "controlled_vehicles": 1,
    "other_vehicles": 1,
    "screen_width": 600,
    "screen_height": 600,
    "centering_position": [0.5, 0.5],
    "scaling": 7,
    "show_trajectories": False,
    "render_agent": True,
    "offscreen_rendering": False
}


env.unwrapped.configure(config)
env.reset()

(array([[[0., 0., 0., 0., 0., 0., 0.],
         [0., 0., 0., 0., 0., 0., 0.],
         [0., 0., 0., 0., 0., 0., 0.],
         [0., 0., 0., 1., 0., 0., 0.],
         [0., 0., 0., 0., 0., 0., 0.],
         [0., 0., 0., 0., 0., 0., 0.],
         [0., 0., 0., 0., 0., 0., 0.]],
 
        [[1., 1., 1., 1., 0., 0., 0.],
         [0., 0., 0., 0., 0., 0., 0.],
         [1., 1., 1., 1., 1., 1., 1.],
         [1., 1., 1., 1., 1., 1., 1.],
         [0., 0., 0., 0., 0., 0., 0.],
         [0., 0., 0., 0., 0., 0., 0.],
         [0., 0., 0., 0., 0., 0., 0.]]], dtype=float32),
 {'speed': 10,
  'crashed': False,
  'action': array([0.42524788], dtype=float32),
  'rewards': {'lane_centering_reward': 1.0,
   'action_reward': 0.42524788,
   'collision_reward': False,
   'on_road_reward': True}})

In [5]:
## Learning with steering and acceleration
TRAIN = True

if True : 
    n_cpu = 6
    batch_size = 64
    model = PPO(
        "MlpPolicy",
        env,
        policy_kwargs=dict(net_arch=[dict(pi=[256, 256], vf=[256, 256])]),
        n_steps=batch_size * 12 // n_cpu,
        batch_size=batch_size,
        n_epochs=10,
        learning_rate=5e-4,
        gamma=0.9,
        verbose=2,
        tensorboard_log="racetrack_ppo/",
    )
    # Train the model
    if TRAIN:
        model.learn(total_timesteps=int(1e5))
        model.save("racetrack_ppo/model_sterring_acceleration")
        del model

Using cpu device
Wrapping the env with a `Monitor` wrapper
Wrapping the env in a DummyVecEnv.


/Users/BOULLIERERWAN1/opt/anaconda3/envs/rl_py311/lib/python3.11/site-packages/stable_baselines3/common/policies.py:486: UserWarning: As shared layers in the mlp_extractor are removed since SB3 v1.8.0, you should now pass directly a dictionary and not a list (net_arch=dict(pi=..., vf=...) instead of net_arch=[dict(pi=..., vf=...)])
  warnings.warn(


Logging to racetrack_ppo/PPO_10
----------------------------
| time/              |     |
|    fps             | 26  |
|    iterations      | 1   |
|    time_elapsed    | 4   |
|    total_timesteps | 128 |
----------------------------
------------------------------------------
| time/                   |              |
|    fps                  | 26           |
|    iterations           | 2            |
|    time_elapsed         | 9            |
|    total_timesteps      | 256          |
| train/                  |              |
|    approx_kl            | 0.0011686706 |
|    clip_fraction        | 0.0195       |
|    clip_range           | 0.2          |
|    entropy_loss         | -1.42        |
|    explained_variance   | -2.11        |
|    learning_rate        | 0.0005       |
|    loss                 | -0.0282      |
|    n_updates            | 10           |
|    policy_gradient_loss | -0.0174      |
|    std                  | 0.996        |
|    value_loss           | 0.0357

In [6]:
import numpy as np
import gymnasium as gym

env = gym.make("racetrack-v0", render_mode="rgb_array")

config = {
    "observation": {
        "type": "OccupancyGrid",
        "features": ["presence", "on_road"],
        "grid_size": [[-18, 18], [-18, 18]],
        "grid_step": [5, 5],
        "as_image": False,
        "align_to_vehicle_axes": True
    },
    "action": {
        "type": "ContinuousAction",
        "longitudinal": True,
        "lateral": True,
        "steering_range": [-np.pi / 4, np.pi / 4],  # [rad]
        "acceleration_range": [-2, 2],  # [m/s²]
        "speed_range": [0, 15],  # [m/s]
    },
    "simulation_frequency": 15,
    "policy_frequency": 5,
    "duration": 300,
    "collision_reward": -1,
    "lane_centering_cost": 4,
    "action_reward": -0.3,
    "controlled_vehicles": 1,
    "other_vehicles": 1,
    "screen_width": 600,
    "screen_height": 600,
    "centering_position": [0.5, 0.5],
    "scaling": 7,
    "show_trajectories": False,
    "render_agent": True,
    "offscreen_rendering": False
}


env.unwrapped.configure(config)
env.reset()

(array([[[0., 0., 0., 0., 0., 0., 0.],
         [0., 0., 0., 0., 0., 0., 0.],
         [0., 0., 0., 0., 0., 0., 0.],
         [0., 0., 0., 1., 0., 0., 0.],
         [0., 0., 0., 0., 0., 0., 0.],
         [0., 0., 0., 0., 0., 0., 0.],
         [0., 0., 0., 0., 0., 0., 0.]],
 
        [[1., 1., 1., 1., 0., 0., 0.],
         [0., 0., 0., 0., 0., 0., 0.],
         [1., 1., 1., 1., 1., 1., 1.],
         [1., 1., 1., 1., 1., 1., 1.],
         [0., 0., 0., 0., 0., 0., 0.],
         [0., 0., 0., 0., 0., 0., 0.],
         [0., 0., 0., 0., 0., 0., 0.]]], dtype=float32),
 {'speed': 10,
  'crashed': False,
  'action': array([-0.17539826], dtype=float32),
  'rewards': {'lane_centering_reward': 1.0,
   'action_reward': 0.17539826,
   'collision_reward': False,
   'on_road_reward': True}})

In [7]:
# Run the algorithm
model = PPO.load("racetrack_ppo/model", env=env)

env = gym.make("racetrack-v0", render_mode="rgb_array")
env = RecordVideo(
    env, video_folder="racetrack_ppo/videos", episode_trigger=lambda e: True
)
env.unwrapped.set_record_video_wrapper(env)

for video in range(10):
    done = truncated = False
    obs, info = env.reset()
    while not (done or truncated):
        # Predict
        action, _states = model.predict(obs, deterministic=True)
        # Get reward
        obs, reward, done, truncated, info = env.step(action)
        # Render
        env.render()
env.close()

Wrapping the env with a `Monitor` wrapper
Wrapping the env in a DummyVecEnv.


ValueError: Observation spaces do not match: Box(-inf, inf, (2, 12, 12), float32) != Box(-inf, inf, (2, 7, 7), float32)